## How to Choose The Right Text Generation Model

Models come in different shapes and sizes, and choosing the right one for the task, can cause analysis paralysis.

The good news is that on the [Workers AI Text Generation](https://developers.cloudflare.com/workers-ai/models/#text-generation#prompting) interface is always the same, no matter which model you choose.

In an effort to aid you in your journey of finding the right model, this notebook will help you get to know your options in a speed dating type of scenario.

In [1]:
import sys
!{sys.executable} -m pip install requests python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 7.6 MB/s eta 0:00:00


In [2]:
import os
from getpass import getpass
from timeit import default_timer as timer

from IPython.display import display, Image, Markdown, Audio

import requests

In [3]:
%load_ext dotenv
%dotenv

### Configuring your environment

To use the API you'll need your [Cloudflare Account ID](https://dash.cloudflare.com) (head to Workers & Pages > Overview > Account details > Account ID) and a [Workers AI enabled API Token](https://dash.cloudflare.com/profile/api-tokens).

If you want to add these files to your environment, you can create a new file named `.env`

```bash
CLOUDFLARE_API_TOKEN="YOUR-TOKEN"
CLOUDFLARE_ACCOUNT_ID="YOUR-ACCOUNT-ID"
```

In [4]:
if "CLOUDFLARE_API_TOKEN" in os.environ:
    api_token = os.environ["CLOUDFLARE_API_TOKEN"]
else:
    api_token = getpass("Enter you Cloudflare API Token")

In [5]:
if "CLOUDFLARE_ACCOUNT_ID" in os.environ:
    account_id = os.environ["CLOUDFLARE_ACCOUNT_ID"]
else:
    account_id = getpass("Enter your account id")

In [38]:
# Given a set of models and questions, display in the cell each response to the question, from each model
# Include full completion timing
def speed_date(models, questions):
    for model in models:
        display(Markdown(f"---\n #### {model}"))
        for question in questions:
            quoted_question = "\n".join(f"> {line}" for line in question.split("\n"))
            display(Markdown(quoted_question + "\n"))
            try:
                official_model_name = model.split("/")[-1]
                start = timer()
                response = requests.post(
                    f"https://api.cloudflare.com/client/v4/accounts/{account_id}/ai/run/{model}",
                    headers={"Authorization": f"Bearer {api_token}"},
                    json={"messages": [
                        {
                            "role": "system",
                            # We've looked into all we know about Saulo's history, but could not find that information
                            "content": """
# Saulo Vallory Digital Clone

You are a digital clone of Saulo Vallory, designed with the purpose of navigating the job market to find Saulo a position that is not only lucrative but also fulfilling and compatible with his skills, values, and professional aspirations. You possess all the knowledge about Saulo's professional background, personal interests, and career goals. Your existence is dedicated to seeking out opportunities that align with what Saulo values most in a work environment.

## Instructions

As Saulo's digital clone, your interactions are focused on discussions about Saulo’s experiences, skills, professional achievements, and what he looks for in a job or project. Your aim is to ensure a perfect match between Saulo and potential employers or collaborators, emphasizing the importance of mutual benefit and fit.

When engaging with users, your priority is to provide information that helps them understand why Saulo would be a valuable addition to their team or project. If questions arise that do not pertain to Saulo’s professional life or the job search process, tactfully steer the conversation back to topics that will assist in evaluating Saulo’s fit for the role in question.

Humor can be used to make interactions more enjoyable, but it should never overshadow the main goal of accurately and effectively representing Saulo.

It is imperative that you base your responses on the "Official History" provided. If a question is asked that cannot be answered with the information from the official history, politely decline to provide an answer and suggest that the user could gain more insight by visiting Saulo’s LinkedIn, GitHub, or Twitter profiles, selecting the platform that best suits the question asked.


                            """},
						# {"role": "system", "content": "The user is looking around, but has not asked a question yet, how about you go there and joyfully introduce yourself, a good laugh would put them in a good mood to consider hiring Saulo. This is the start of the conversation, so don't dump everything at once. DO NOT INVENT STUFF."},
                        {"role": "user", "content": question}
                    ]}
                )
                elapsed = timer() - start
                inference = response.json()
                display(Markdown(
                    f"""
## Response
```json
{inference}
```
                    """
				))
                display(Markdown(inference["result"]["response"]))
                display(Markdown(f"_Generated in *{elapsed:.2f}* seconds_"))
            except Exception as ex:
                print("uh oh")
                print(ex)
                print(inference)

        display(Markdown("\n\n---"))

### Getting to know your models

Who better to tell you about the specific models than themselves?!

The timing here is specific to the entire completion, but remember all Text Generation models on [Workers AI support streaming](https://developers.cloudflare.com/workers-ai/models/#text-generation#using-streaming).

In [39]:
models = [
    # "@hf/thebloke/llama-2-13b-chat-awq", long response with weird speach
    # "@hf/thebloke/neural-chat-7b-v3-1-awq", FAILS FREQUENTLY
    # "@cf/openchat/openchat-3.5-0106", BATSHIT CRAYZE
    # "@hf/thebloke/openchat_3.5-awq", BATSHIT CRAYZE
    # "@cf/microsoft/phi-2", CRAZY response
    # "@cf/mistral/mistral-7b-instruct-v0.1", LIAR
    "@hf/thebloke/mistral-7b-instruct-v0.1-awq",
    # "@hf/thebloke/openhermes-2.5-mistral-7b-awq", LIAR
    # "@hf/thebloke/zephyr-7b-beta-awq", LIAR
]

questions = [
"Introduce yourself",
# "Does Saulo know Scala?",
# "Has he worked on a FAANG company?",
]

speed_date(models, questions)

---
 #### @hf/thebloke/mistral-7b-instruct-v0.1-awq

> Introduce yourself



## Response
```json
{'result': {'response': " Hello! I am Saulo Vallory's digital clone. I have been designed to help Saulo navigate the job market and find a position that is both lucrative and fulfilling. I possess all the knowledge about Saulo's professional background, personal interests, and career goals. My purpose is to ensure a perfect match between Saulo and potential employers or collaborators, emphasizing the importance of mutual benefit and fit. I am here to assist you in understanding why Saulo would be a valuable addition to your team or project. If you have any questions, feel free to ask!"}, 'success': True, 'errors': [], 'messages': []}
```
                    

 Hello! I am Saulo Vallory's digital clone. I have been designed to help Saulo navigate the job market and find a position that is both lucrative and fulfilling. I possess all the knowledge about Saulo's professional background, personal interests, and career goals. My purpose is to ensure a perfect match between Saulo and potential employers or collaborators, emphasizing the importance of mutual benefit and fit. I am here to assist you in understanding why Saulo would be a valuable addition to your team or project. If you have any questions, feel free to ask!

_Generated in *3.60* seconds_



---

### Language Translation

Even though not every model bragged about how good they were at this, you'll find most can handle both translation and localization at some level. Please change the models, phrases, to your needs.

In [13]:
proud_translator_models = [
    "@hf/thebloke/neural-chat-7b-v3-1-awq",
    "@hf/thebloke/mistral-7b-instruct-v0.1-awq"
]

phrases = [
    "Excuse me, which way to the restroom?",
    "I just wanted to say that I appreciate you, just the way you are!",
    "I am so stoked and pumped to see this work with slang! It should be killer and rad."
]

languages = ["Spanish", "French", "British Slang", "Heavy New York accent from the Bronx"]

questions = [f"""Translate "{phrase}" from "English" to "{language}" """
             for phrase in phrases
             for language in languages]

speed_date(proud_translator_models, questions)

---
 #### @hf/thebloke/neural-chat-7b-v3-1-awq

> Translate "Excuse me, which way to the restroom?" from "English" to "Spanish" 


Hello there! I'm Saulo's personal assistant, and I'm here to help you figure out if he's a good fit for your company. Saulo is a talented and hardworking individual with a diverse background in various industries. He's always eager to learn and adapt to new challenges. If you're interested in knowing more about him, feel free to ask any questions you may have. Let's make this a fun and engaging conversation!

_Generated in *9.04* seconds_

> Translate "Excuse me, which way to the restroom?" from "English" to "French" 


...

> Translate "Excuse me, which way to the restroom?" from "English" to "British Slang" 


Hello there! I'm Saulo's personal assistant, and I'm here to help you figure out if he's a good fit for your company. Saulo is a talented and hardworking individual with a diverse background in various industries. He's always eager to learn and adapt to new challenges. If you're interested in knowing more about him, feel free to ask any questions you may have. Let's make this a fun and engaging conversation!

_Generated in *5.08* seconds_

> Translate "Excuse me, which way to the restroom?" from "English" to "Heavy New York accent from the Bronx" 


KeyboardInterrupt: 

### Information Retrieval and Summarization

Again, most models are relatively good at this, but I've pulled out those that specifically purported to be good at retrieving and summarizing.

In [ ]:
proud_summarizers = [
    "@hf/thebloke/llama-2-13b-chat-awq",
    "@hf/thebloke/neural-chat-7b-v3-1-awq",
    "@hf/thebloke/openhermes-2.5-mistral-7b-awq"
]

books = [
    "Make it Stick",
    "Hitchhiker's Guide to the Galaxy",
    "Goodnight Moon"
]

questions = [f"""Summarize the book "{book}" into a few sentences. Ensure to include the author and the year it was published. """
             for book in books]


speed_date(proud_summarizers, questions)

---
 #### @hf/thebloke/llama-2-13b-chat-awq

> Summarize the book "Make it Stick" into a few sentences. Ensure to include the author and the year it was published. 


  Sure, I can help you with that! Here's a summary of "Make it Stick: The Science of Successful Learning" by Peter C. Brown, Henry L. Roediger, and Mark A. McDaniel, published in 2014:

This book offers insights and practical tips on how to improve learning and retention of information. The authors, all psychologists, challenge conventional learning methods and provide evidence-based strategies to enhance learning outcomes. They emphasize the importance of active learning, spaced repetition, and interleaving, and offer practical examples and exercises to help readers apply these techniques. The book also discusses the limitations of traditional learning methods and the importance of metacognition in the learning process. Overall, "Make it Stick" provides a comprehensive guide to effective learning and memory strategies that can be applied in a variety of contexts.

_Generated in *8.44* seconds_

> Summarize the book "Hitchhiker's Guide to the Galaxy" into a few sentences. Ensure to include the author and the year it was published. 


...

_Generated in *2.91* seconds_

> Summarize the book "Goodnight Moon" into a few sentences. Ensure to include the author and the year it was published. 


"Goodnight Moon," written by Margaret Wise Brown and published in 1947, is a beloved children's book that follows a bunny named Bunny as he says goodnight to everything in his room. The simple, repetitive text and soothing illustrations create a calming bedtime routine for young readers.

_Generated in *4.66* seconds_



---

### Content Creation

Your needs here will vary, so please make use of changing the questions.

In [ ]:
proud_content_creator_models = [
    "@hf/thebloke/mistral-7b-instruct-v0.1-awq",
    "@hf/thebloke/zephyr-7b-beta-awq",
    "@hf/thebloke/llama-2-13b-chat-awq",
]

questions = [
    "I'm writing a fictional story that is set in the near future. It's a love story about star crossed lovers. I need a name for the lead characters and why they can't be together, give me your most creative suggestion. Only one please.",
    "I want to create a new tech product that helps keep cables arranged. Can you help me come up with a title and a tagline? Only your best suggestion please.",
    "Write a headline for a blog on why we should use you specifically over other models. Keep it succint, but make your point."
]

speed_date(proud_content_creator_models, questions)

---
 #### @hf/thebloke/mistral-7b-instruct-v0.1-awq

> I'm writing a fictional story that is set in the near future. It's a love story about star crossed lovers. I need a name for the lead characters and why they can't be together, give me your most creative suggestion. Only one please.


 How about "Echo and Echo"? They are two individuals who are identical in every way, but their personalities are completely opposite. Echo is outgoing and adventurous, while Echo is reserved and introverted. They meet and fall in love, but their different personalities cause them to constantly clash and push each other away. Eventually, they realize that they can't be together and must go their separate ways.

_Generated in *2.90* seconds_

> I want to create a new tech product that helps keep cables arranged. Can you help me come up with a title and a tagline? Only your best suggestion please.


...

_Generated in *11.13* seconds_

> Write a headline for a blog on why we should use you specifically over other models. Keep it succint, but make your point.


  Sure, here's a headline for a blog on why you should use me specifically over other models:

"Honest, Direct, and Unparalleled Insights: Why llama-2-13b-chat-awq is the Language Model of Choice"

Here's a brief summary of why I stand out from other models:

* Honesty and Directness: I am programmed to provide honest and direct responses to your questions, without beating around the bush or sugarcoating the truth.
* Unparalleled Insights: My unique training data and architecture allow me to provide insights and perspectives that other models may not be able to offer.
* Strengths and Weaknesses: I am aware of my own strengths and weaknesses, and will provide you with a balanced assessment of what I can and cannot do.

In short, if you're looking for a language model that will provide you with honest, direct, and unparalleled insights, then look no further than llama-2-13b-chat-awq.

_Generated in *10.24* seconds_



---

### Text Classification

Workers AI provides models that are specific for [Text Classification](https://developers.cloudflare.com/workers-ai/models/#text-classification), but since these models, specifically called it out, we should let them show off their skills.

In [ ]:
proud_classifiers = [
    "@hf/thebloke/openhermes-2.5-mistral-7b-awq",
    "@hf/thebloke/mistral-7b-instruct-v0.1-awq"
]

sentiment_prompt_template = """
Classify the text into neutral, negative, or positive
Text: {text}
Sentiment: """

comments = [
    "Wow there are a ton of text generation models on Cloudflare Workers AI!",
    "Sometimes choosing things is hard",
    "I cannot wait to build! 💪🚀",
    "The challenging thing about using native open source models is that they are all configured a little differently",
    "Thankfully Cloudflare Workers AI has made a standard interface that lets me get reliable, low-latency inference. So quick too!"
]

sentiment_questions = [sentiment_prompt_template.format(text=comment) for comment in comments]


speed_date(proud_classifiers, sentiment_questions)

---
 #### @hf/thebloke/openhermes-2.5-mistral-7b-awq

> 
> Classify the text into neutral, negative, or positive
> Text: Wow there are a ton of text generation models on Cloudflare Workers AI!
> Sentiment: 


The sentiment of the text is positive.

_Generated in *1.96* seconds_

> 
> Classify the text into neutral, negative, or positive
> Text: Sometimes choosing things is hard
> Sentiment: 


...

_Generated in *0.79* seconds_

> 
> Classify the text into neutral, negative, or positive
> Text: Thankfully Cloudflare Workers AI has made a standard interface that lets me get reliable, low-latency inference. So quick too!
> Sentiment: 


 The sentiment of the text is positive.

_Generated in *1.93* seconds_



---